# Raw Data Exploration

This notebook explores the original Excel data files to understand the structure and content of each sheet.
We will analyze:
- Structure (number of rows/columns)
- Relevant columns per sheet
- Sample content for entity detection or regular expression use


In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
from pathlib import Path


# Load both Excel files from data/raw
reporting_path = Path("../raw/Reporting_Inventory.xlsx")
glossary_path = Path("../raw/Business_Glossary.xlsx")

reporting_data = pd.read_excel(reporting_path, sheet_name=None)
glossary_data = pd.read_excel(glossary_path, sheet_name=None)

## Overview of Reporting Inventory Sheets

In [2]:
# Display full content in cells (not truncated)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

# Summary of each sheet in the reporting inventory
reporting_summary = {
    sheet: {
        "rows": df.shape[0],
        "columns": df.shape[1],
        "columns_names": list(df.columns)
    }
    for sheet, df in reporting_data.items()
}
pd.DataFrame.from_dict(reporting_summary, orient='index')

,rows,columns,columns_names
Reporting Glossary,32,4,"[Unnamed: 0, INDEX REPORTING INVENTORY , Unnamed: 2, Unnamed: 3]"
Reports,309,31,"[guid, ID Data Product, Name, Description, Type of data product, BI Portal Section, Area, Department, Team, Team Product Owner, Category according to use, Release date, Current version, Status, Closing date, Transferred to, Refresh frequency, Main Data Sources, Sensitivity, RLS, Type of RLS, GDPR, Technology, Support material, Product Owner, Data Owner, Data Steward, Access, Monthly Effort (h), PBIX_File, ID_Data_Product_Dupl]"
Views Glossary,17,4,"[Unnamed: 0, INDEX VIEWS INVENTORY , Unnamed: 2, Unnamed: 3]"
Views,1486,15,"[ID Data Product, Report Name, Product Owner, PBIX_File, Report View, Description, Category, Status, Rename, Dimensions, KPIs, Other Terms, Filters, Tags, Priority]"
"Areas, Departments & Teams",63,5,"[Area, Deparment, Team, Department Leader, Team Leader]"
LookUp,7,3,"[Views Category, Unnamed: 1, Views Status]"


### Reports sheet

In [3]:
# Extract the 'Reports' sheet as a separate DataFrame
reports_df = reporting_data["Reports"]

# Display basic info
print(f"Total rows: {reports_df.shape[0]} | Total columns: {reports_df.shape[1]}")

# Show the first few rows
reports_df.head(2)


Total rows: 309 | Total columns: 31


,guid,ID Data Product,Name,Description,Type of data product,BI Portal Section,Area,Department,Team,Team Product Owner,Category according to use,Release date,Current version,Status,Closing date,Transferred to,Refresh frequency,Main Data Sources,Sensitivity,RLS,Type of RLS,GDPR,Technology,Support material,Product Owner,Data Owner,Data Steward,Access,Monthly Effort (h),PBIX_File,ID_Data_Product_Dupl
0,(ID Purview),RPPBI0001,Laundry Cost Analysis Report 2024,"Report focused on the deep analysis of cost imputations from Laundry account, providing inisight analysis regarding material cost and cleaning activity performance.",Report,BI for Operations & Global Transformation;,Operations & Global Transformation,Operations,Room Division,Operational Excellence Analytics,Operative,2024-03-10,v2.0,Historical,2025-01-09,NaN,Monthly,Google Cloud;SAP Business Objects;TMS;Server F;,Organization;,No,NaN,NaN,Power BI,Contains: User Guide and Documentation,Kelly Hunter,Larry Moore,NaN,BI Portal,0.0,LetterReport.pbix,RPPBI0001
1,(ID Purview),RPPBI0002,Laundry Cost Analysis Report 2023,"Report focused on the deep analysis of cost imputations from Laundry account, providing inisight analysis regarding material cost and cleaning activity performance.",Report,BI for Operations & Global Transformation;,Operations & Global Transformation,Operations,Room Division,Operational Excellence Analytics,Operative,2024-03-10,v2.0,Historical,2024-03-10,NaN,Monthly,Google Cloud;SAP Business Objects;TMS;Server F;,Organization;,No,NaN,NaN,Power BI,Contains: User Guide and Documentation,Kelly Hunter,Larry Moore,NaN,BI Portal,0.0,HotReport.pbix,RPPBI0002


In [4]:
# Extract the glossary sheet to map field descriptions
glossary_df = reporting_data["Reporting Glossary"]

# Build a dictionary of field -> description (cleaning columns)
glossary_df = glossary_df.rename(columns={
    glossary_df.columns[1]: "field",
    glossary_df.columns[2]: "description"
})
field_descriptions = glossary_df.set_index("field")["description"].to_dict()

# Build a diagnostic table
field_diagnostics = []

for col in reports_df.columns:
    col_data = reports_df[col]
    null_count = col_data.isnull().sum()
    unique_count = col_data.nunique(dropna=True)
    sample_values = col_data.dropna().astype(str).value_counts().head(5).to_dict()
    
    field_diagnostics.append({
        "field": col,
        "description": field_descriptions.get(col, "No description in glossary"),
        "null_count": null_count,
        "null_percentage": round(null_count / len(col_data) * 100, 2),
        "unique_values": unique_count,
        "top_5_values": sample_values
    })

# Convert to DataFrame for easy viewing
field_diagnostics_df = pd.DataFrame(field_diagnostics).sort_values(by="null_percentage", ascending=False)
field_diagnostics_df.reset_index(drop=True, inplace=True)

# Display the summary
field_diagnostics_df.head(32)


,field,description,null_count,null_percentage,unique_values,top_5_values
0,Type of RLS,The type of RLS restriction applied.,306,99.03,2,"{'HQ-All, BU-All, HotelSpecific-Hotel, Other-None': 2, 'HQ-All, BU-All': 1}"
1,Transferred to,No description in glossary,264,85.44,31,"{'RPPBI0142': 5, 'RPPBI0168': 4, 'RPPBI0122': 3, 'RPPBI0021': 2, 'RPPBI0133': 2}"
2,Data Steward,"Member that acts as a Data Steward (with strong knowledge of how to apply business rules in reporting and ensure data quality). This could be an individual or group that defines the standards for the data catalog. A data steward is responsible for maintaining nomenclature, data quality standards, security controls, compliance requirements, and rules for the assigned object.",256,82.85,13,"{'Matthew Dennis': 16, 'Kelly Hunter': 7, 'Elizabeth Lynch': 6, 'Tasha Hall': 4, 'Tyler Stephens': 4}"
3,GDPR,Wether it contains GDPR sensitive data,217,70.23,3,"{'No': 65, 'Yes': 26, 'YES': 1}"
4,Support material,"Localization of documentation for users, refreshment manual, technical documentation for BI to facilitate future handovers.",217,70.23,4,"{'Contains: User Guide': 43, 'Contains: User Guide and Documentation': 32, 'Contains: Documentation': 14, 'Contains: Glossary for Certificates': 3}"
5,Closing date,No description in glossary,203,65.70,57,"{'2024-01-07 00:00:00': 19, '2025-01-07 00:00:00': 7, '2023-07-15 00:00:00': 5, '2025-01-01 00:00:00': 5, '2024-04-01 00:00:00': 4}"
6,Data Owner,Business member with extensive knowledge of the insights provided by the report. Owners are often senior executives or business area owners that define governance or business processes over certain data areas.,187,60.52,24,"{'Jonathan Shields': 17, 'Larry Moore': 14, 'Ronald Cole': 14, 'Cynthia White': 12, 'Christopher Chung': 8}"
7,Release date,No description in glossary,174,56.31,74,"{'2025-01-01': 17, '2023-01-01': 8, '2024-09-01': 6, '2024-01-01': 6, '2022-03-01': 4}"
8,Monthly Effort (h),Estimated manual effort (in hours) to update and validate the data product.,164,53.07,14,"{'0.0': 34, '0.5': 25, '1.0': 23, '2.0': 20, '30.0': 13}"
9,Current version,No description in glossary,87,28.16,12,"{'v1.0': 132, 'v2.0': 52, 'v3.0': 15, 'v4.0': 6, 'v2.1': 3}"


### Views sheet

In [5]:
# Extract the 'Views' sheet as a separate DataFrame
views_df = reporting_data["Views"]

# Display basic info
print(f"Total rows: {views_df.shape[0]} | Total columns: {views_df.shape[1]}")

# Show the first few rows
views_df.head(2)


Total rows: 1486 | Total columns: 15


,ID Data Product,Report Name,Product Owner,PBIX_File,Report View,Description,Category,Status,Rename,Dimensions,KPIs,Other Terms,Filters,Tags,Priority
0,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,CRITERIA,Methodolody and definition of the algorithim of Feeder Market,Informative,Productive,NaN,NaN,NaN,NaN,NaN,NaN,Priority 1
1,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,DESTINATION_OF_FEEDER_MARKETS,View focused on understand the performance by hotel for a specific feeder market o selection of feeder marktes.,Functional,Productive,NaN,"Hotel, month, Feeder Market, Segment, Channel Mix, Room Type","Total Revenue, Room Revenue, RN, Lead Time, Lenght of Stay, AOV, ADR, ADR Net, %Cost",NaN,NaN,NaN,Priority 1


In [6]:
# Extract the glossary sheet for views
views_glossary_df = reporting_data["Views Glossary"]

# Clean and rename glossary columns to map field descriptions
views_glossary_df = views_glossary_df.rename(columns={
    views_glossary_df.columns[1]: "field",
    views_glossary_df.columns[2]: "description"
})
views_field_descriptions = views_glossary_df.set_index("field")["description"].to_dict()

# Build the diagnostic summary for Views
views_diagnostics = []

for col in views_df.columns:
    col_data = views_df[col]
    null_count = col_data.isnull().sum()
    unique_count = col_data.nunique(dropna=True)
    sample_values = col_data.dropna().astype(str).value_counts().head(5).to_dict()
    
    views_diagnostics.append({
        "field": col,
        "description": views_field_descriptions.get(col, "No description in glossary"),
        "null_count": null_count,
        "null_percentage": round(null_count / len(col_data) * 100, 2),
        "unique_values": unique_count,
        "top_5_values": sample_values
    })

# Convert to DataFrame and display
views_diagnostics_df = pd.DataFrame(views_diagnostics).sort_values(by="null_percentage", ascending=False)
views_diagnostics_df.reset_index(drop=True, inplace=True)

# Show the summary
views_diagnostics_df.head(20)


,field,description,null_count,null_percentage,unique_values,top_5_values
0,Rename,New name of the Report View,1481,99.66,5,"{'Internal Surveys': 1, 'Executive Summary': 1, 'Online Ratings': 1, 'P&L Summary Tables': 1, 'Commercial Approach': 1}"
1,Other Terms,"List of other interesting terms such as specific and important values of dimensions, acronyms that offer extra information, etc.",1367,91.99,51,"{'eCommerce, OTA, Digital': 19, 'Actuals, Predicted;Forecast, TY, LY, LY2, NY': 8, 'Gross, FInancial, Net, Net Commercial, Fair': 7, 'CCG, CGW, CRO B2B Digital, NHPro Agencies, NHPro Companies, NHPro Meetings C&M, NHPro Meetings Form': 4, 'B2B Digital, Booking, CRO B2B Digital, CRO Web, Digital, eCommerce, Expedia, OTA, Web': 4}"
2,Tags,Relevant keywords related to page view.,1311,88.22,78,"{'Hotel Masterfile': 13, 'Benchmark, BPC, USALI, P&L': 11, 'OCI, Check-In': 8, 'Performance, eCommerce': 6, 'Performance, OTA': 5}"
3,Filters,Filters applied at page view level.,1277,85.94,32,"{'No': 83, 'Hotel_ID is not (blank)': 26, 'It has been excluded some hotels as: Casino de Madrid, BBVA, hotels from brasil...': 22, 'Hotel_Status_TY is OPEN': 11, 'BU & Sub BU; Month; Hotel; Contract & Consolidation; Country&City, Comparable; Contract; TOP GOP; Brand; RRM; DRM; RM;OpsRegional;RC': 7}"
4,KPIs,List of main KPIs of the Report View,1203,80.96,151,"{'Total Revenue, Room Revenue, RN, Lead Time, Lenght of Stay, AOV, ADR, ADR Net, %Cost': 11, 'EBITDA, GOP, Revenues, Expenses, BPC Accounts, Drivers, Totals, Ratios': 11, 'Total Revenue, Room Revenue, RN': 8, 'Total Revenue': 8, '% Occupancy': 8}"
5,Dimensions,List of main dimensions of the Report View,1191,80.15,97,"{'Hotel; Hotel_ID; Hotel_Brand; Hotel_BU; Hotel_SubBU; Hotel_Country; QUEST Scope; Operations Regional; Month; Hotel ID': 38, 'BU, Sub BU, Country, City, Name, Brand, Certification, OPS Regional, Comparable, Contract, Eng_MNT_Manager, Consolidation_Mngt': 27, 'Channel, Subchannel, BU, Country, Brand, Segment, Feeder Market, City, Hotel Name, Stay Month, Subsegment, Segment Type': 15, 'BU, Sub BU, Country, City, Name, Brand, OPS Regional, Comparable, Contract, Eng_MNT_Manager, Consolidation_Mngt, Consolidated': 9, 'Region, BU, SubBU, Country, City, Hotel, OPS Regional, Brand, Stars, Quality Team, Date, USALI, Department, Center of Cost, CECO, EFT': 9}"
6,Status,Stage of the Data Product,975,65.61,4,"{'Productive': 427, 'Internal': 69, 'Discontinued': 10, 'Renamed': 5}"
7,Description,Brief description of the Report View of the data product.,922,62.05,379,"{'Index page with interactive buttons to other views.': 43, 'Currently not in use. It remains as hiden until it's deleted permanently.': 6, 'Focus on Corporate Business (Corporate Sales & Airlines). Special focus on Account Segmentation.': 4, 'Index': 4, 'This view offers a vision of reservation behaviour, with focus on specific KPIs such that Lead Time, Lenght of Stay, AOV or Cancellation Rate. The KPIs can be analyzed by different dimensions, and also are shown in a dynamic table that allow users to select the dimensions needed.': 4}"
8,Category,Category of the Report View,505,33.98,7,"{'Functional': 532, 'Index': 192, 'Executive': 130, 'Informative': 75, 'Self-Service': 25}"
9,Priority,Priority to inventory and define the Reports View.,2,0.13,7,"{'Priority 4': 391, 'Priority 1': 366, 'Priority 2': 299, 'Priority 3': 217, 'Priority 9': 204}"


### Export to CSV for API Use

In [7]:
# 1. Columns to include in the final DataFrame used by the API
output_columns = ["ID Data Product", "Report Name", "Report View","Tags"]

# 2. Columns used to generate the 'keywords' field
keyword_sources = ["Tags"]

# Load only the output columns initially
report_views_df = reporting_data["Views"][output_columns].copy()
report_views_df = report_views_df[report_views_df["ID Data Product"].notnull()]

# Access the full source sheet for additional keyword columns
full_views_df = reporting_data["Views"]

# Function to generate 'keywords' from selected source columns
def build_keywords(row_id):
    words = []
    row = full_views_df.loc[row_id]
    for col in keyword_sources:
        if col in row and pd.notna(row[col]):
            value = row[col]
            if col == "Tags":
                items = str(value).replace(";", ",").split(",")
            else:
                items = str(value).split()
            words.extend(item.strip() for item in items if len(item.strip()) > 2)
    return ", ".join(sorted(set(words)))

# Apply keyword generation using row index to access all columns
report_views_df["keywords"] = report_views_df.index.map(build_keywords)

# Display result
report_views_df.head(100)


,ID Data Product,Report Name,Report View,Tags,keywords
0,RPPBI0032,Feeder Market - 2024,CRITERIA,NaN,
1,RPPBI0032,Feeder Market - 2024,DESTINATION_OF_FEEDER_MARKETS,NaN,
2,RPPBI0032,Feeder Market - 2024,EXECUTIVE VIEW,NaN,
3,RPPBI0032,Feeder Market - 2024,FEEDER MARKET FLOWS,NaN,
4,RPPBI0032,Feeder Market - 2024,FEEDER_MARKET_DETAIL,NaN,
5,RPPBI0032,Feeder Market - 2024,FEEDER_MARKETS_OF_DESTINATION,NaN,
6,RPPBI0032,Feeder Market - 2024,MENU,NaN,
7,RPPBI0032,Feeder Market - 2024,OE MARKET INSIGHTS,NaN,
8,RPPBI0032,Feeder Market - 2024,TARGETS FOLLOW UP,NaN,
9,RPPBI0154,Feeder Market - 2025,CRITERIA,NaN,


In [29]:
from pathlib import Path

# Define the target path for the API data directory
api_output_path = Path("../api")
api_output_path.mkdir(parents=True, exist_ok=True)  # Create the folder if it doesn't exist

# Define the output CSV file path
csv_path = api_output_path / "reports.csv"

# Export the DataFrame to CSV (no index)
report_views_df.to_csv(csv_path, index=False)

print(f"CSV file saved at: {csv_path}")


CSV file saved at: ../api/reports.csv


## Overview of Business Glossary Sheets

In [30]:
# Summary of each sheet in the business glossary
glossary_summary = {
    sheet: {
        "rows": df.shape[0],
        "columns": df.shape[1],
        "columns_names": list(df.columns)
    }
    for sheet, df in glossary_data.items()
}
pd.DataFrame.from_dict(glossary_summary, orient='index')

,rows,columns,columns_names
Consolidated,1855,74,"[Nick Name, Name, Definition, IsDefinitionRichText, Status, Related Terms, Synonyms, Acronym, Experts, Stewards, Resources, Parent Term Name, Term Template Names, [Attribute][Data Office template]Asset type, [Attribute][Data Office template]Calculation, [Attribute][Data Office template]Domain, [Attribute][Data Office template]GDPR, [Attribute][Data Office template]Scope, [Attribute][Data Office template]Subdomain, [Attribute][Controlling and Strategic Planning]Asset type, [Attribute][Controlling and Strategic Planning]Calculation, [Attribute][Controlling and Strategic Planning]Domain, [Attribute][Controlling and Strategic Planning]GDPR, [Attribute][Controlling and Strategic Planning]Scope, [Attribute][Controlling and Strategic Planning]Subdomain, [Attribute][FB template]Asset type, [Attribute][FB template]Calculation, [Attribute][FB template]Domain, [Attribute][FB template]GDPR, [Attribute][FB template]Scope, [Attribute][FB template]Subdomain, [Attribute][Finance template]Asset type, [Attribute][Finance template]Calculation, [Attribute][Finance template]Domain, [Attribute][Finance template]GDPR, [Attribute][Finance template]Scope, [Attribute][Finance template]Subdomain, [Attribute][General Secretary template]Asset type, [Attribute][General Secretary template]Calculation, [Attribute][General Secretary template]Domain, [Attribute][General Secretary template]GDPR, [Attribute][General Secretary template]Scope, [Attribute][General Secretary template]Subdomain, [Attribute][Marketing template]Asset type, [Attribute][Marketing template]Calculation, [Attribute][Marketing template]Domain, [Attribute][Marketing template]GDPR, [Attribute][Marketing template]Scope, [Attribute][Marketing template]Subdomain, [Attribute][Operations and Global Transformation]Asset type, [Attribute][Operations and Global Transformation]Calculation, [Attribute][Operations and Global Transformation]Domain, [Attribute][Operations and Global Transformation]GDPR, [Attribute][Operations and Global Transformation]Scope, [Attribute][Operations and Global Transformation]Subdomain, [Attribute][People and Sustainable Business template]Asset type, [Attribute][People and Sustainable Business template]Calculation, [Attribute][People and Sustainable Business template]Domain, [Attribute][People and Sustainable Business template]GDPR, [Attribute][People and Sustainable Business template]Scope, [Attribute][People and Sustainable Business template]Subdomain, [Attribute][Commercial template]Asset type, [Attribute][Commercial template]Calculation, [Attribute][Commercial template]Domain, [Attribute][Commercial template]GDPR, [Attribute][Commercial template]Scope, [Attribute][Commercial template]Subdomain, Area, Domain, Subdomain, GDPR, Calculation, Asset type, Data Owner]"
Datamap,182,8,"[Area, Domain, Subdomain, Name, Role, Experts, Tier, Informed]"
Informed,35,3,"[Name, Mail, Training]"
DataStewards,16,5,"[Subdomain, NameDS, RoleDS, Steward, InformedDS]"
DataOffice,23,19,"[Nick Name, Name, Definition, IsDefinitionRichText, Status, Related Terms, Synonyms, Acronym, Experts, Stewards, Resources, Parent Term Name, Term Template Names, [Attribute][Data Office template]Asset type, [Attribute][Data Office template]Calculation, [Attribute][Data Office template]Domain, [Attribute][Data Office template]GDPR, [Attribute][Data Office template]Scope, [Attribute][Data Office template]Subdomain]"
Controlling,898,19,"[Nick Name, Name, Definition, IsDefinitionRichText, Status, Related Terms, Synonyms, Acronym, Experts, Stewards, Resources, Parent Term Name, Term Template Names, [Attribute][Controlling and Strategic Planning]Asset type, [Attribute][Controlling and Strategic Planning]Calculation, [Attribute][Controlling and Strategic Planning]Domain, [Attribute][Controlling and Strategic Planning]GDPR, [Attribute][Controlling and Strategic Planning]Scope, [Attribute][Controlling and Strategic Planning]Subdomain]"
FB,45,19,"[Nick Name, Name, Definiti